## Importing Data and Libraries

In [1]:
import pandas as pd
import numpy as np
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax
from scipy.spatial import distance
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score

ModuleNotFoundError: No module named 'pyfume'

In [ ]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

stats = pd.read_csv('stats.csv')
stats.columns = ['var', 'mean', 'std']

X_train = Train.drop('output', axis=1)
y_train = Train['output']

X_test = Test.drop('output', axis=1)
y_test = Test['output']

maxs = X_train.max().tolist()
mins = X_train.min().tolist()

var_names = X_train.columns.to_list()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
savecols = X_train.columns

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.array
y_test = y_test.array

## Building the model

### Clustering

In this section, we will try out different numbers of clusters in order to minimize the separation between clusters. 

#### Functions

In [ ]:
def SeparationMetric(X_train, clust_centers, part_matrix):
    metric = 0
    for clust in range(clust_centers.shape[0]):
        for point in range(X_train.shape[0]):
            metric += part_matrix[point, clust] * distance.euclidean(X_train[point], clust_centers[clust])
    return metric

#def SeparationMetric(X_train, clust_centers, part_matrix):
#    silhouette_score(X_train, )
#    return metric

In [ ]:
# Since pyfume's clustering algorithm was using the the target, I made some modifcations to their fcm
# code as seen below

def fcm(data, n_clusters, m=2, max_iter=1000, error=0.005):
        #data: 2d array, size (N, S). N is the number of instances; S is the number of variables
        #n_clusters: number of clusters
        #m: fuzzy clustering coefficient
        #max_it: maximum number of iterations, default=1000
        #error: stopping criterion, default=0.005
        #seed: seed for random initialization of u matrix
        
    n_instances = data.shape[0]
        
    #randomly initaliaze u
    np.random.seed(1231241421)
    u = np.random.rand(n_instances, n_clusters)
    u = np.fmax(u, np.finfo(np.float64).eps)
    ut = u.T
        
    for it in range(0,max_iter):
        #copy old u matrix
        u_old = ut.copy()
        u_old /= np.ones((n_clusters, 1)).dot(np.atleast_2d(u_old.sum(axis=0)))
        u_old = np.fmax(u_old, np.finfo(np.float64).eps)
        
        #elevate to m
        um = u_old ** m
        
        #calculate cluster centers
        centers = um.dot(data) / (np.ones((data.shape[1], 1)).dot(np.atleast_2d(um.sum(axis=1))).T)
        
        #calculate distances
        dist = cdist(centers, data, metric='euclidean')
        dist = np.fmax(dist, np.finfo(np.float64).eps)
        
        #calculate objective
        jm = (um * dist ** 2).sum()
    
        #calculate new u matrix
        ut = dist ** (- 2. / (m - 1))
        ut /= np.ones((n_clusters, 1)).dot(np.atleast_2d(ut.sum(axis=0)))
    
        #stopping criterion
        if np.linalg.norm(ut - u_old) < error:
            break
    
    partition_matrix = ut.T
    return centers, partition_matrix

In [ ]:
def Cluster(nclusters):
    clust_centers, part_matrix = fcm(X_train, n_clusters = nclusters)
    return clust_centers, part_matrix

#### Code

In [ ]:
max_clusters = 2
min_metric = np.inf

for i in range(max_clusters):
    i += 1
    clust_centers, part_matrix = Cluster(i)
    metric = SeparationMetric(X_train, clust_centers, part_matrix)
    if metric < min_metric:
        min_metric = metric
        best_number = i

print(f'Best number of clusters: {best_number}')

In [ ]:
clust_centers, part_matrix = Cluster(best_number)
clust_centers.shape, part_matrix.shape

### Building the model

In [ ]:
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()

ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms()

modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
model = modbuilder.get_model()

### Inference and Metrics

In [ ]:
modtester = SugenoFISTester(model, X_test, var_names)
y_pred_probs = clip(modtester.predict()[0], 0, 1)
y_pred_probs = column_stack((1 - y_pred_probs, y_pred_probs))
y_pred = argmax(y_pred_probs,axis=1)

In [ ]:
acc_score = accuracy_score(y_test, y_pred)
print("Accuracy: {:.3f}".format(acc_score))
rec_score = recall_score(y_test, y_pred)
print("Recall: {:.3f}".format(rec_score))
prec_score = precision_score(y_test, y_pred)
print("Precision Score: {:.3f}".format(prec_score))
F1_score = f1_score(y_test, y_pred)
print("F1-Score: {:.3f}".format(F1_score))
kappa = cohen_kappa_score(y_test, y_pred)
print("Kappa Score: {:.3f}".format(kappa))

## Model Visualization/Interpretation

In [ ]:
model._lvs['chol'] # example of how pyFUME defines une linguistic variable in simpful

In [ ]:
model.get_rules() # print the rules associated with the model

In [ ]:
# The pyFUME library doesn't atribute a universe of discourse to the linguistic variables it creates in simpful. In order to fix this, we set each universe considering the maximum
# value heald by each dataframe column. This has to be done since, otherwise, the plotting functions won't work.

for ix in range(len(var_names)):
    max = maxs[ix]
    min = mins[ix]
    uod = [min, max]
    model._lvs[var_names[ix]]._universe_of_discourse = uod


In [ ]:
model.produce_figure("",2)

In [ ]:
def FunctionsToZ(model, stats):
    for f in model._outputfunctions:
        expression = model._outputfunctions[f]
        item_list = expression.split('+')
        for k in range(len(item_list) - 1):
            item_list[k] = item_list[k].split('*')
            values = stats[stats['var'] == item_list[k][1]]
            item_list[k][0] = float(item_list[k][0]) * values.iloc[0,2] + values.iloc[0,1]
        finalfunct = ''
        for sum in item_list:
            if len(sum) != 1: finalfunct = finalfunct + str(sum[0]) + '*' + sum[1] + '+'
        finalfunct = finalfunct[:-4] + str(item_list[-1])
        model._outputfunctions[f] = finalfunct
        return model

In [ ]:
vars(model)

In [ ]:
def ClusterstoZ(model, stats):
    for lv in model._lvs:
        data = stats[stats['var'] == lv]
        mean = data.iloc[0,1]
        std = data.iloc[0,2]
        dict = model._lvs[lv]._FSlist[0]._funpointer
        dict._mu = 140#dict._mu - mean  #confirmar
        dict._sigma = dict._sigma + std - 1#confirmar
        model._lvs[lv]._FSlist[0]._funpointer = dict
    return model

In [ ]:
def UODtoZ(model, stats):
    for lv in model._lvs:
        data = stats[stats['var'] == lv]
        mean = data.iloc[0,1]
        std = data.iloc[0,2]
        uod = model._lvs[lv]._universe_of_discourse
        for i in range(2):
            uod[i] = uod[i] * std + mean
        model._lvs[lv]._universe_of_discourse = uod
    return model

In [ ]:
modelZ = FunctionsToZ(model, stats)
modelZ = ClusterstoZ(modelZ, stats)
modelZ = UODtoZ(modelZ, stats)

In [ ]:
modelZ.produce_figure("",2)

In [ ]:
vars(modelZ._lvs['chol']._FSlist[0]._funpointer)